In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession,Row,functions
spark=SparkSession.builder.appName('SparkSQL040').getOrCreate()
sc=spark.sparkContext

In [3]:
file_ratings_100K='/home/ggomarr/Documents/Education/Udemy_Spark/ml-100k/u.data'
file_info_100K='/home/ggomarr/Documents/Education/Udemy_Spark/ml-100k/u.item'

In [49]:
def process_movie_rating_100K(movie_row):
    movie=movie_row.split()
    return Row(movie_id=int(movie[1]),
               rating=float(movie[2]))

def build_movie_dict_100K(file_nom):
    movie_dict={}
    with open(file_nom) as f:
        for movie_row in f:
            movie=movie_row.split('|')
            movie_dict[int(movie[0])]=movie[1]
    return movie_dict

In [50]:
movie_dict=build_movie_dict_100K(file_info_100K)

from pyspark.sql.types import StringType
title_grabber=functions.udf(lambda movie_id: movie_dict[movie_id],StringType())

In [41]:
for key in movie_dict.keys()[:10]:
    print('{:4d} - {}'.format(key,movie_dict[key]))

   1 - Toy Story (1995)
   2 - GoldenEye (1995)
   3 - Four Rooms (1995)
   4 - Get Shorty (1995)
   5 - Copycat (1995)
   6 - Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
   7 - Twelve Monkeys (1995)
   8 - Babe (1995)
   9 - Dead Man Walking (1995)
  10 - Richard III (1995)


In [5]:
movies_rdd=sc.textFile(file_ratings_100K).map(process_movie_rating_100K)
movies_df=spark.createDataFrame(movies_rdd)

In [6]:
movies_df.show(5)

+--------+------+
|movie_id|rating|
+--------+------+
|     242|   3.0|
|     302|   3.0|
|     377|   1.0|
|      51|   2.0|
|     346|   1.0|
+--------+------+
only showing top 5 rows



In [54]:
pop_movies=movies_df.groupBy('movie_id').count()

(pop_movies.withColumn('title',title_grabber(mean_and_count['movie_id']))
           .orderBy('count',ascending=False)
           .show(5,False))

+--------+-----+-------------------------+
|movie_id|count|title                    |
+--------+-----+-------------------------+
|50      |583  |Star Wars (1977)         |
|258     |509  |Contact (1997)           |
|100     |508  |Fargo (1996)             |
|181     |507  |Return of the Jedi (1983)|
|294     |485  |Liar Liar (1997)         |
+--------+-----+-------------------------+
only showing top 5 rows



In [32]:
(movies_df.groupBy('movie_id')
          .agg(functions.mean(movies_df['rating']).alias('mean_rating'))
          .orderBy('mean_rating',ascending=False)
          .show(5))

+--------+-----------+
|movie_id|mean_rating|
+--------+-----------+
|    1189|        5.0|
|    1536|        5.0|
|    1122|        5.0|
|    1599|        5.0|
|     814|        5.0|
+--------+-----------+
only showing top 5 rows



In [51]:
mean_and_count=(movies_df.groupBy('movie_id')
                         .agg({'rating':'mean','movie_id':'count'})
                         .withColumnRenamed('avg(rating)','rating')
                         .withColumnRenamed('count(movie_id)','count'))             

(mean_and_count.withColumn('title',title_grabber(mean_and_count['movie_id']))
               .filter(mean_and_count['count']>10)
               .orderBy('rating',ascending=False)
               .show(5,False))

+--------+-----------------+-----+------------------------------------------------------+
|movie_id|rating           |count|title                                                 |
+--------+-----------------+-----+------------------------------------------------------+
|408     |4.491071428571429|112  |Close Shave, A (1995)                                 |
|318     |4.466442953020135|298  |Schindler's List (1993)                               |
|169     |4.466101694915254|118  |Wrong Trousers, The (1993)                            |
|483     |4.45679012345679 |243  |Casablanca (1942)                                     |
|114     |4.447761194029851|67   |Wallace & Gromit: The Best of Aardman Animation (1996)|
+--------+-----------------+-----+------------------------------------------------------+
only showing top 5 rows

